In [1]:
# define a road segment class corresponding to the original filtered shape file
class RoadSegInr(object):
    def __init__(self, tmc, road_num, shape_length):
        self.tmc = tmc
        self.road_num = road_num
        self.shape_length = shape_length

In [2]:
# define a road segment capacity class corresponding to the capacity file
class RoadSegCapac(object):
    def __init__(self, road_invent, length, route_num, AB_AM_capac, \
                 AB_MD_capac, AB_PM_capac, AB_NT_capac):
        self.road_invent = road_invent
        self.length = length
        self.route_num = route_num
        self.AB_AM_capac = AB_AM_capac
        self.AB_MD_capac = AB_MD_capac
        self.AB_PM_capac = AB_PM_capac
        self.AB_NT_capac = AB_NT_capac

In [3]:
# define a lookup class corresponding to the lookup table file
class LookUp(object):
    def __init__(self, road_inv_ID, tmc):
        self.road_inv_ID = road_inv_ID
        self.tmc = tmc

In [4]:
##### read in raw data

import openpyxl

data_folder = '/home/jzh/Dropbox/Research/\
Data-driven_estimation_inverse_optimization/INRIX/Raw_data/'

# load filtered INRIX attribute table raw data
wb_INRIX = openpyxl.load_workbook(data_folder + 'filtered_INRIX_attribute_table.xlsx')

# load filtered capacity attribute table raw data
wb_capac = openpyxl.load_workbook(data_folder + 'filtered_capacity_attribute_table.xlsx')

# load lookup table raw data
wb_lookup = openpyxl.load_workbook(data_folder + 'roadinv_id_to_tmc_lookup.xlsx') 

# get sheet name from workbook
sheet_INRIX_name = wb_INRIX.sheetnames[0].encode('utf-8')
sheet_capac_name = wb_capac.sheetnames[0].encode('utf-8')
sheet_lookup_name = wb_lookup.sheetnames[0].encode('utf-8')

# get sheet of filtered INRIX attribute table raw data
sheet_INRIX = wb_INRIX.get_sheet_by_name(sheet_INRIX_name)

# get sheet of filtered capacity attribute table raw data
sheet_capac = wb_capac.get_sheet_by_name(sheet_capac_name)

# get sheet of lookup table raw data
sheet_lookup = wb_lookup.get_sheet_by_name(sheet_lookup_name)

In [5]:
##### extract attributes of interest from INRIX sheet

tmc_list = []
road_num_list = []
shape_length_list = []

for i in xrange(2, 1 + sheet_INRIX.max_row):
    tmc_list.append(sheet_INRIX.cell(row=i, column=2).value.encode('utf-8'))
    road_num_list.append(sheet_INRIX.cell(row=i, column=3).value.encode('utf-8'))
    shape_length_list.append(sheet_INRIX.cell(row=i, column=13).value)

assert(len(tmc_list) == len(road_num_list) and \
       len(road_num_list) == len(shape_length_list))    

In [6]:
##### extract attributes of interest from capacity sheet

road_invent_list = []
length_list = []
route_num_list = []
AB_AM_capac_list = [] 
AB_MD_capac_list = []
AB_PM_capac_list = []
AB_NT_capac_list = []

for i in xrange(2, 1 + sheet_capac.max_row):
    road_invent_list.append(sheet_capac.cell(row=i, column=26).value) 
    length_list.append(sheet_capac.cell(row=i, column=2).value)
    route_num_list.append(sheet_capac.cell(row=i, column=8).value)
    AB_AM_capac_list.append(sheet_capac.cell(row=i, column=18).value)
    AB_MD_capac_list.append(sheet_capac.cell(row=i, column=20).value)
    AB_PM_capac_list.append(sheet_capac.cell(row=i, column=22).value)
    AB_NT_capac_list.append(sheet_capac.cell(row=i, column=24).value)

assert(len(road_invent_list) == len(length_list) and \
               len(length_list) == len(route_num_list) and \
               len(route_num_list) == len(AB_AM_capac_list) and \
               len(AB_AM_capac_list) == len(AB_MD_capac_list) and \
               len(AB_MD_capac_list) == len(AB_PM_capac_list) and \
               len(AB_PM_capac_list) == len(AB_NT_capac_list))    

In [7]:
##### extract attributes of interest from lookup sheet

road_inv_ID_lookup_list = []
tmc_lookup_list = []

for i in xrange(2, 1 + sheet_lookup.max_row):
    road_inv_ID_lookup_list.append(sheet_lookup.cell(row=i, column=1).value) 
    tmc_lookup_list.append(str(sheet_lookup.cell(row=i, column=4).value))

assert(len(road_inv_ID_lookup_list) == len(tmc_lookup_list))    

In [8]:
# instantiation of RoadSegInr class
road_seg_inr = RoadSegInr(tmc_list, road_num_list, shape_length_list)

# instantiation of RoadSegCapac class
road_seg_capac = RoadSegCapac(road_invent_list, length_list, route_num_list, \
                              AB_AM_capac_list, AB_MD_capac_list, \
                              AB_PM_capac_list, AB_NT_capac_list)

# instantiation of LookUp class
look_up = LookUp(road_inv_ID_lookup_list, tmc_lookup_list)

In [9]:
# define a derived road segment class containing the capacity info
class RoadSegInrCapac(RoadSegInr):
    def __init__(self, tmc, road_num, shape_length, AB_AM_capac, \
                 AB_MD_capac, AB_PM_capac, AB_NT_capac):
        RoadSegInr.__init__(self, tmc, road_num, shape_length)
        self.AB_AM_capac = AB_AM_capac
        self.AB_MD_capac = AB_MD_capac
        self.AB_PM_capac = AB_PM_capac
        self.AB_NT_capac = AB_NT_capac

In [10]:
# make a dictionary from look_up
tmc_roadInv_dict = {i:j for (i, j) in zip(look_up.tmc, look_up.road_inv_ID)}

# make dictionaries from road_seg_capac
roadInv_capac_dict_AM = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_AM_capac)}
roadInv_capac_dict_MD = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_MD_capac)}
roadInv_capac_dict_PM = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_PM_capac)}
roadInv_capac_dict_NT = {i:j for (i, j) in zip(road_seg_capac.road_invent, \
                                               road_seg_capac.AB_NT_capac)}

In [11]:
capac_AM = []
capac_MD = []
capac_PM = []
capac_NT = []

for i in range(len(road_seg_inr.tmc)):
    capac_AM.append(roadInv_capac_dict_AM[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_MD.append(roadInv_capac_dict_MD[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_PM.append(roadInv_capac_dict_PM[tmc_roadInv_dict[road_seg_inr.tmc[i]]])
    capac_NT.append(roadInv_capac_dict_NT[tmc_roadInv_dict[road_seg_inr.tmc[i]]])

In [12]:
# instantiation of RoadSegInrCapac class
road_seg_inr_capac = RoadSegInrCapac(tmc_list, road_num_list, shape_length_list, \
                                     capac_AM, capac_MD, capac_PM, capac_NT)